# **PAGE RANK**



***Summarization*** adalah suatu cara untuk meringkas suatu dokumen ke dalam bentuk yang lebih sederhana dengan mengambil poin-poin penting dalam dokumen tersebut.

Secara umum, pendekatan news summarization dibagi menjadi dua, yaitu extractive dan abstractive. Extractive summarization berarti mengidentifikasi bagian-bagian penting dari teks dan membuat kalimat dari kata-kata tersebut. Sementara abstractive summarization mereproduksi materi penting dengan cara baru setelah melakukan interpretasi dan memeriksa teks menggunakan NLP untuk membuat kalimat baru yang berasal dari dokumen aslinya.




***TextRank*** adalah algoritma peringkat berbasis grafik untuk memproses teks. TextRank menghasilkan ekstraksi kalimat sebagai ringkasan. Salah satu kelebihan dari algoritma ini, tidak diperlukannya pelatihan menggunakan data training pada algoritma yang digunakan.

**Langkah pertama adalah meng-import library yang diperlukan**

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

**Teks berita yang digunakan**

In [2]:
text='''The Double Asteroid Redirection Test successfully changed the trajectory of the asteroid Dimorphos when the NASA spacecraft intentionally slammed into the space rock on September 26, according to the agency. The DART mission, a full-scale demonstration of deflection technology, was the world’s first conducted on behalf of planetary defense. The mission was also the first time humanity intentionally changed the motion of a celestial object in space. Prior to impact, it took Dimorphos 11 hours and 55 minutes to orbit its larger parent asteroid Didymos. Astronomers used ground-based telescopes to measure how Dimorphos’ orbit changed after impact. Now, it takes Dimorphos 11 hours and 23 minutes to circle Didymos. The DART spacecraft changed the moonlet asteroid’s orbit by 32 minutes. Initially, astronomers expected DART to be a success if it shortened the trajectory by 10 minutes. “All of us have a responsibility to protect our home planet. After all, it’s the only one we have,” said NASA Administrator Bill Nelson. “This mission shows that NASA is trying to be ready for whatever the universe throws at us. NASA has proven we are serious as a defender of the planet. This is a watershed moment for planetary defense and all of humanity, demonstrating commitment from NASA’s exceptional team and partners from around the world.”'''

In [3]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Tokenizing paragraph ke bentuk list sentences**

In [4]:
sentences=sent_tokenize(text)
print(sentences)

['The Double Asteroid Redirection Test successfully changed the trajectory of the asteroid Dimorphos when the NASA spacecraft intentionally slammed into the space rock on September 26, according to the agency.', 'The DART mission, a full-scale demonstration of deflection technology, was the world’s first conducted on behalf of planetary defense.', 'The mission was also the first time humanity intentionally changed the motion of a celestial object in space.', 'Prior to impact, it took Dimorphos 11 hours and 55 minutes to orbit its larger parent asteroid Didymos.', 'Astronomers used ground-based telescopes to measure how Dimorphos’ orbit changed after impact.', 'Now, it takes Dimorphos 11 hours and 23 minutes to circle Didymos.', 'The DART spacecraft changed the moonlet asteroid’s orbit by 32 minutes.', 'Initially, astronomers expected DART to be a success if it shortened the trajectory by 10 minutes.', '“All of us have a responsibility to protect our home planet.', 'After all, it’s the 

**Tahap Preprocessing**

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True



*   re.sub(r'[^\w\s]') menghapus semua tanda baca dari setiap kalimat menggunakan pemahaman daftar
*   Hapus stopwords (a, the, an, dll) untuk fokus pada kata-kata yang lebih signifikan dalam sebuah kalimat.




In [6]:
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
print(sentence_tokens)

[['double', 'asteroid', 'redirection', 'test', 'successfully', 'changed', 'trajectory', 'asteroid', 'dimorphos', 'nasa', 'spacecraft', 'intentionally', 'slammed', 'space', 'rock', 'september', '26', 'according', 'agency'], ['dart', 'mission', 'fullscale', 'demonstration', 'deflection', 'technology', 'worlds', 'first', 'conducted', 'behalf', 'planetary', 'defense'], ['mission', 'also', 'first', 'time', 'humanity', 'intentionally', 'changed', 'motion', 'celestial', 'object', 'space'], ['prior', 'impact', 'took', 'dimorphos', '11', 'hours', '55', 'minutes', 'orbit', 'larger', 'parent', 'asteroid', 'didymos'], ['astronomers', 'used', 'groundbased', 'telescopes', 'measure', 'dimorphos', 'orbit', 'changed', 'impact'], ['takes', 'dimorphos', '11', 'hours', '23', 'minutes', 'circle', 'didymos'], ['dart', 'spacecraft', 'changed', 'moonlet', 'asteroids', 'orbit', '32', 'minutes'], ['initially', 'astronomers', 'expected', 'dart', 'success', 'shortened', 'trajectory', '10', 'minutes'], ['us', 'res

Seperti yang kita ketahui kita perlu mengkonversi data teks ke dalam bentuk numerik, pertama kita akan menghitung penyisipan kata & menggunakan penyematan ini akan menghitung penyisipan kalimat menggunakan variabel sentence_tokens yang dihitung di atas.

- Menggunakan Word2Vec() gensim, menghitung embedding untuk setiap kata dari teks.
- Menggunakan pemahaman daftar, kami akan mengganti token kata dengan penyematannya di kalimat_tokens
- Menghitung max_length kalimat dalam teks yang ada
- Penyematan kalimat padding menggunakan 0s hingga max_len. Ini telah dilakukan untuk mempertahankan ukuran embedding yang sama untuk setiap kalimat.

In [7]:
w2v=Word2Vec(sentence_tokens,size=1,min_count=1,iter=1000)
sentence_embeddings=[[w2v[word][0] for word in words] for words in sentence_tokens]
max_len=max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
print(sentence_embeddings)

[array([1.2918975, 1.5915023, 1.424116 , 1.4121064, 1.4467098, 1.5382317,
       1.5134743, 1.5915023, 1.5365793, 1.6182544, 1.5082136, 1.5764458,
       1.4246264, 1.6471946, 1.5734042, 1.6188911, 1.7296653, 1.6748086,
       1.7022004], dtype=float32), array([1.5909445, 1.6882219, 1.5302439, 1.6292546, 1.7472122, 1.7504741,
       1.6810927, 1.6839105, 1.6468395, 1.7119603, 1.7967135, 1.7317495,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       ], dtype=float32), array([1.6882219, 1.3961402, 1.6839105, 1.4614487, 1.6892772, 1.5764458,
       1.5382317, 1.4816146, 1.5066162, 1.4804983, 1.6471946, 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       ], dtype=float32), array([1.3100374, 1.4721438, 1.4949446, 1.5365793, 1.5497124, 1.5401931,
       1.4178144, 1.5086985, 1.4931173, 1.3899549, 1.3405302, 1.5915023,
       1.562259 , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       ],

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


**calculate the similarity matrix**


> Inisialisasi kesamaan_matriks dimensi N x N di mana N adalah jumlah total kalimat dalam teks

> Menggunakan 1-spatial.distance.cosine(), hitung kesamaan antara setiap dua pasang kalimat.





In [8]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
print(similarity_matrix)

[[1.         0.77043754 0.73066145 0.79750407 0.65673184 0.61752486
  0.61498106 0.65762484 0.47216785 0.5250662  0.65711278 0.47036043
  0.79860079]
 [0.77043754 1.         0.95183975 0.95394218 0.85319388 0.80501884
  0.80297899 0.85399824 0.62667412 0.69473588 0.85390836 0.62544829
  0.95834243]
 [0.73066145 0.95183975 1.         0.90454125 0.90091938 0.85113281
  0.85250568 0.89994073 0.68359745 0.74823987 0.90215069 0.68440574
  0.90794176]
 [0.79750407 0.95394218 0.90454125 1.         0.83000332 0.78411967
  0.78065282 0.83219987 0.61477208 0.6814009  0.83173227 0.61299634
  0.99794602]
 [0.65673184 0.85319388 0.90091938 0.83000332 1.         0.94014323
  0.94137853 0.99765056 0.73189008 0.80741876 0.99942011 0.73087609
  0.80988145]
 [0.61752486 0.80501884 0.85113281 0.78411967 0.94014323 1.
  0.99626654 0.94175869 0.7821306  0.8632592  0.94173282 0.77898991
  0.7650243 ]
 [0.61498106 0.80297899 0.85250568 0.78065282 0.94137853 0.99626654
  1.         0.9386161  0.7969842  0.874

**implement PageRank**


> Convert the similarity matrix to a network/graph


> Apply pagerank()





In [9]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)
print(scores)

{0: 0.06595837702198956, 1: 0.07816115951855858, 2: 0.07980752701016651, 3: 0.07737513992280871, 4: 0.0809080710255183, 5: 0.08009710495505575, 6: 0.08029171935865889, 7: 0.08117429487446932, 8: 0.07170589442096058, 9: 0.0754495539976037, 10: 0.08104802658809455, 11: 0.0716030497213804, 12: 0.07642008158473519}


Sekarang, yang akan kita lakukan adalah membuat kamus dengan kalimat (kita dapatkan setelah sent_tokenize()) sebagai kunci & PageRank mereka sebagai nilai & mengurutkan kamus ini berdasarkan PageRanks kalimat. Disini diambil 4 kalimat teratas untuk ringkasan:

In [10]:
top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
print(top)

{'Initially, astronomers expected DART to be a success if it shortened the trajectory by 10 minutes.': 0.08117429487446932, '“This mission shows that NASA is trying to be ready for whatever the universe throws at us.': 0.08104802658809455, 'Astronomers used ground-based telescopes to measure how Dimorphos’ orbit changed after impact.': 0.0809080710255183, 'The DART spacecraft changed the moonlet asteroid’s orbit by 32 minutes.': 0.08029171935865889}


In [11]:
for sent in sentences:
    if sent in top.keys():
        print(sent)

Astronomers used ground-based telescopes to measure how Dimorphos’ orbit changed after impact.
The DART spacecraft changed the moonlet asteroid’s orbit by 32 minutes.
Initially, astronomers expected DART to be a success if it shortened the trajectory by 10 minutes.
“This mission shows that NASA is trying to be ready for whatever the universe throws at us.


# **SENTIMEN ANALISIS**

# **Data crawling**
Data crawling atau perayapan data adalah proses pengambilan data yang tersedia secara online untuk umum. Proses ini kemudian mengimpor informasi atau data yang telah ditemukan ke dalam file lokal di komputer Anda. 

## Twint

Twint adalah sebuah tools yang digunakan untuk melakukan scrapping dari aplikasi twitter yang disetting secara khusus menggunakan bahasa pemrograman Python. Twint dapat kita gunakan dan jalankan tanpa harus menggunakan API dari Twitter itu sendiri, dengan kapasitas scrapping data maksimalnya adalah 3200 tweet.

Bukan hanya digunakan pada tweet, twint juga bisa kita gunakan untuk melakukan scrapping pada user, followers, retweet dan sebagainya. Twint memanfaatkan operator pencarian twitter untuk memungkinkan proses penghapusan tweet dari user tertentu, memilih dan memilah informasi-informasi yang sensitif, termasuk email dan nomor telepon di dalamnya.

In [12]:
!git clone --depth=1 https://github.com/twintproject/twint.git
%cd twint
!pip3 install . -r requirements.txt

Cloning into 'twint'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 3), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.
/content/twint
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/twint
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 263 kB 14.9 MB/s 
     |████████████████████████████████| 385 kB 56.7 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |██████████████████████████

In [13]:
!pip install nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
!pip install aiohttp==3.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 10.3 MB/s 
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 4.0.2
    Uninstalling async-timeout-4.0.2:
      Successfully uninstalled async-timeout-4.0.2
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.8.3
    Uninstalling aiohttp-3.8.3:
      Successfully uninstalled aiohttp-3.8.3


In [15]:
import nest_asyncio
nest_asyncio.apply() #digunakan sekali untuk mengaktifkan tindakan serentak dalam notebook jupyter.
import twint #untuk import twint
c = twint.Config()
c.Search = 'tragedi kanjuruhan'
c.Pandas = True
c.Limit = 70
twint.run.Search(c)

1597746823753789446 2022-11-30 00:18:22 +0000 <SaifuddinSay> @CNNIndonesia Akibat muka tembok pengurus PSSI di Tragedi Kanjuruhan.
1597740242978865152 2022-11-29 23:52:13 +0000 <yusrilmiqball> Loh 2 desember udah mulai aja liga ya. Tapi dari kubu federasi kok lempeng lempeng aja ya hehehe   selama belum ada tindakan yang serius terkait tragedi kanjuruhan, hanya satu kata untuk sepakbola Indonesia BOIKOT!! SHAME ON YOU @PSSI @pt_lib 🖕🖕
1597736514263601152 2022-11-29 23:37:24 +0000 <kategoriUMUM> BANGSAT KALIAN PARA PELAKU TRAGEDI KANJURUHAN 🖕🏻
1597733030835421184 2022-11-29 23:23:33 +0000 <SuaraPersiba> Ini Tanggapan 2 Pemain Persiba Atas Tragedi Berdarah di Stadion Kanjuruhan -  https://t.co/se9xxd3YLV  https://t.co/UasXBcWCGw
1597721879670067200 2022-11-29 22:39:15 +0000 <robhittur> @Liga1Match2022 Saya rasa statement sikap @Arie_Kriting di salah satu podcast adalah sikap yg harus diambil oleh seluruh supporter Indonesia, yakni untuk tidak menonton liga Indonesia sampai usut tuntas ke

In [16]:
Tweets_dfs = twint.storage.panda.Tweets_df
Tweets_dfs["tweet"]

0     @CNNIndonesia Akibat muka tembok pengurus PSSI...
1     Loh 2 desember udah mulai aja liga ya. Tapi da...
2      BANGSAT KALIAN PARA PELAKU TRAGEDI KANJURUHAN 🖕🏻
3     Ini Tanggapan 2 Pemain Persiba Atas Tragedi Be...
4     @Liga1Match2022 Saya rasa statement sikap @Ari...
                            ...                        
75    Kejati Minta Tersangka Baru, Pasal Kelalaian D...
76    Dengan berbagai kerumitan di dunia persepakbol...
77    Asisten Kapolri bidang Operasi (Asops) Irjen A...
78    265 Anak Korban Tragedi Kanjuruhan Dapat Penda...
79    "Kevia Naswa, Penyintas Tragedi Kanjuruhan yan...
Name: tweet, Length: 80, dtype: object

In [17]:
Tweets_dfs["tweet"].to_csv("TRAGEDI.csv")

# **Mengambil Data**

Proses ini digunakan untuk mengambil 100 data tweet yang telah disimpan dalam github dengan format .csv



In [18]:
#install library pandas
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
#install library numpy
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import pandas as pd 
import numpy as np

data_abstrak = pd.read_csv("TRAGEDI.csv")

data_abstrak

,Unnamed: 0,tweet
0,0,@CNNIndonesia Akibat muka tembok pengurus PSSI...
1,1,Loh 2 desember udah mulai aja liga ya. Tapi da...
2,2,BANGSAT KALIAN PARA PELAKU TRAGEDI KANJURUHAN 🖕🏻
3,3,Ini Tanggapan 2 Pemain Persiba Atas Tragedi Be...
4,4,@Liga1Match2022 Saya rasa statement sikap @Ari...
...,...,...
75,75,"Kejati Minta Tersangka Baru, Pasal Kelalaian D..."
76,76,Dengan berbagai kerumitan di dunia persepakbol...
77,77,Asisten Kapolri bidang Operasi (Asops) Irjen A...
78,78,265 Anak Korban Tragedi Kanjuruhan Dapat Penda...


In [21]:
#install library sastrawi
!pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 13.4 MB/s 


In [22]:
#install library swifter
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 830 kB 15.2 MB/s 
     |████████████████████████████████| 280 kB 76.1 MB/s 
     |████████████████████████████████| 1.6 MB 21.7 MB/s 
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=7d46c493f6d935237c630a5782e01fea417dc723b1517455e698f4a4dbda0075
  Stored in directory: /root/.cache/pip/wheels/29/a7/0e/3a8f17ac69d759e1e93647114bc9bdc95957e5b0cbfd405205
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


# *Case Folding**

Tahap untuk merubah teks yang memiliki huruf kapital menjadi huruf kecil

In [23]:
data_abstrak['tweet'] = data_abstrak['tweet'].str.lower()


data_abstrak['tweet']

0     @cnnindonesia akibat muka tembok pengurus pssi...
1     loh 2 desember udah mulai aja liga ya. tapi da...
2      bangsat kalian para pelaku tragedi kanjuruhan 🖕🏻
3     ini tanggapan 2 pemain persiba atas tragedi be...
4     @liga1match2022 saya rasa statement sikap @ari...
                            ...                        
75    kejati minta tersangka baru, pasal kelalaian d...
76    dengan berbagai kerumitan di dunia persepakbol...
77    asisten kapolri bidang operasi (asops) irjen a...
78    265 anak korban tragedi kanjuruhan dapat penda...
79    "kevia naswa, penyintas tragedi kanjuruhan yan...
Name: tweet, Length: 80, dtype: object

In [24]:
#install library nltk
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Menghapus Karakter Spesial

In [25]:
import string 
import re #regex library
# import word_tokenize & FreqDist from NLTK

from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\'," ").replace('\\f'," ").replace('\\r'," ")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_special)
data_abstrak['tweet']


0     akibat muka tembok pengurus pssi di tragedi ka...
1     loh 2 desember udah mulai aja liga ya. tapi da...
2      bangsat kalian para pelaku tragedi kanjuruhan ??
3     ini tanggapan 2 pemain persiba atas tragedi be...
4     saya rasa statement sikap _kriting di salah sa...
                            ...                        
75    kejati minta tersangka baru, pasal kelalaian d...
76    dengan berbagai kerumitan di dunia persepakbol...
77    asisten kapolri bidang operasi (asops) irjen a...
78    265 anak korban tragedi kanjuruhan dapat penda...
79    "kevia naswa, penyintas tragedi kanjuruhan yan...
Name: tweet, Length: 80, dtype: object

## Menghapus Angka

In [26]:

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_number)
data_abstrak['tweet']

0     akibat muka tembok pengurus pssi di tragedi ka...
1     loh  desember udah mulai aja liga ya. tapi dar...
2      bangsat kalian para pelaku tragedi kanjuruhan ??
3     ini tanggapan  pemain persiba atas tragedi ber...
4     saya rasa statement sikap _kriting di salah sa...
                            ...                        
75    kejati minta tersangka baru, pasal kelalaian d...
76    dengan berbagai kerumitan di dunia persepakbol...
77    asisten kapolri bidang operasi (asops) irjen a...
78     anak korban tragedi kanjuruhan dapat pendampi...
79    "kevia naswa, penyintas tragedi kanjuruhan yan...
Name: tweet, Length: 80, dtype: object

## Menghapus Tanda Baca

In [27]:

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_punctuation)
data_abstrak['tweet']

0     akibat muka tembok pengurus pssi di tragedi ka...
1     loh  desember udah mulai aja liga ya tapi dari...
2        bangsat kalian para pelaku tragedi kanjuruhan 
3     ini tanggapan  pemain persiba atas tragedi ber...
4     saya rasa statement sikap kriting di salah sat...
                            ...                        
75    kejati minta tersangka baru pasal kelalaian da...
76    dengan berbagai kerumitan di dunia persepakbol...
77    asisten kapolri bidang operasi asops irjen agu...
78     anak korban tragedi kanjuruhan dapat pendampi...
79    kevia naswa penyintas tragedi kanjuruhan yang ...
Name: tweet, Length: 80, dtype: object

## Menghapus Spasi


In [28]:

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_whitespace_LT)


#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_whitespace_multiple)
data_abstrak['tweet']

0     akibat muka tembok pengurus pssi di tragedi ka...
1     loh desember udah mulai aja liga ya tapi dari ...
2         bangsat kalian para pelaku tragedi kanjuruhan
3     ini tanggapan pemain persiba atas tragedi berd...
4     saya rasa statement sikap kriting di salah sat...
                            ...                        
75    kejati minta tersangka baru pasal kelalaian da...
76    dengan berbagai kerumitan di dunia persepakbol...
77    asisten kapolri bidang operasi asops irjen agu...
78    anak korban tragedi kanjuruhan dapat pendampin...
79    kevia naswa penyintas tragedi kanjuruhan yang ...
Name: tweet, Length: 80, dtype: object

## Menghapus huruf 

In [29]:

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", " ", text)

data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_singl_char)
data_abstrak['tweet']

0     akibat muka tembok pengurus pssi di tragedi ka...
1     loh desember udah mulai aja liga ya tapi dari ...
2         bangsat kalian para pelaku tragedi kanjuruhan
3     ini tanggapan pemain persiba atas tragedi berd...
4     saya rasa statement sikap kriting di salah sat...
                            ...                        
75    kejati minta tersangka baru pasal kelalaian da...
76    dengan berbagai kerumitan di dunia persepakbol...
77    asisten kapolri bidang operasi asops irjen agu...
78    anak korban tragedi kanjuruhan dapat pendampin...
79    kevia naswa penyintas tragedi kanjuruhan yang ...
Name: tweet, Length: 80, dtype: object

In [30]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# **Tokenizing**

Tokenizing adalah proses pemisahan teks menjadi potongan-potongan yang disebut sebagai token untuk kemudian di analisa. Kata, angka, simbol, tanda baca dan entitas penting lainnya dapat dianggap sebagai token

In [31]:

# NLTK word Tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data_abstrak['tweet'] = data_abstrak['tweet'].apply(word_tokenize_wrapper)
data_abstrak['tweet']

0     [akibat, muka, tembok, pengurus, pssi, di, tra...
1     [loh, desember, udah, mulai, aja, liga, ya, ta...
2     [bangsat, kalian, para, pelaku, tragedi, kanju...
3     [ini, tanggapan, pemain, persiba, atas, traged...
4     [saya, rasa, statement, sikap, kriting, di, sa...
                            ...                        
75    [kejati, minta, tersangka, baru, pasal, kelala...
76    [dengan, berbagai, kerumitan, di, dunia, perse...
77    [asisten, kapolri, bidang, operasi, asops, irj...
78    [anak, korban, tragedi, kanjuruhan, dapat, pen...
79    [kevia, naswa, penyintas, tragedi, kanjuruhan,...
Name: tweet, Length: 80, dtype: object

# **Filtering(Stopwords Removal)**
Proses untuk menghapus kata hubung atau kata yang tidak memiliki makna


In [32]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:

list_stopwords = stopwords.words('indonesian')

#Menghapus Stopword dari list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data_abstrak['tweet'] = data_abstrak['tweet'].apply(stopwords_removal)

data_abstrak['tweet']

0     [akibat, muka, tembok, pengurus, pssi, tragedi...
1     [loh, desember, udah, aja, liga, ya, kubu, fed...
2                [bangsat, pelaku, tragedi, kanjuruhan]
3     [tanggapan, pemain, persiba, tragedi, berdarah...
4     [statement, sikap, kriting, salah, podcast, si...
                            ...                        
75    [kejati, tersangka, pasal, kelalaian, tragedi,...
76    [kerumitan, dunia, persepakbolaan, indonesia, ...
77    [asisten, kapolri, bidang, operasi, asops, irj...
78    [anak, korban, tragedi, kanjuruhan, pendamping...
79    [kevia, naswa, penyintas, tragedi, kanjuruhan,...
Name: tweet, Length: 80, dtype: object

# **Stemming**
Data training hasil dari filtering akan dilakukan pengecekan atau pencarian kata-kata yang sesuai dengan kamus umum. Apabila data training hasil filtering sesuai dengan kamus umum maka kata akan dikeluarkan sementara, karena sudah dianggap sebagai kata dasar. Apabila masih terdapat kata yang tidak termasuk dalam kata dasar maka tahap selanjutnya adalah menghapus inflection suffixes yang merupakan akhiran pertama. Kata yang memiliki akhiran partticles seperti “-pun”, “-kah”, “-tah”, “- lah” dan akhiran possessive pronoun seperti “-mu”, “-ku” dan “-nya” dihilangkan. Setelah dilakukan proses case folding, tokenezing, dan filtering, proses selanjutnya yaitu stemming. Stemming yang digunakan pada penelitian ini menggunakan algoritma Enhanced Confix Stipping Stemmer, terdiri dari beberapa langkah: Data training hasil dari filtering akan dilakukan pengecekan atau pencarian kata-kata yang sesuai dengan kamus umum. Apabila data training hasil filtering sesuai dengan kamus umum maka kata akan dikeluarkan sementara, karena sudah dianggap sebagai kata dasar. Apabila masih terdapat kata yang tidak termasuk dalam kata dasar maka tahap selanjutnya adalah menghapus inflection suffixes yang merupakan akhiran pertama. Kata yang memiliki akhiran partticles seperti “-pun”, “-kah”, “-tah”, “- lah” dan akhiran possessive pronoun seperti “-mu”, “-ku” dan “-nya” dihilangkan.

In [34]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data_abstrak['tweet']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data_abstrak['tweet'] = data_abstrak['tweet'].swifter.apply(get_stemmed_term)
data_abstrak['tweet']

505
------------------------
akibat : akibat
muka : muka
tembok : tembok
pengurus : urus
pssi : pssi
tragedi : tragedi
kanjuruhan : kanjuruhan
loh : loh
desember : desember
udah : udah
aja : aja
liga : liga
ya : ya
kubu : kubu
federasi : federasi
lempeng : lempeng
hehehe : hehehe
tindakan : tindak
serius : serius
terkait : kait
sepakbola : sepakbola
indonesia : indonesia
boikot : boikot
shame : shame
on : on
you : you
lib : lib
bangsat : bangsat
pelaku : laku
tanggapan : tanggap
pemain : main
persiba : persiba
berdarah : darah
stadion : stadion
statement : statement
sikap : sikap
kriting : kriting
salah : salah
podcast : podcast
yg : yg
diambil : ambil
supporter : supporter
menonton : tonton
usut : usut
tuntas : tuntas
keadilan : adil
korban : korban
shg : shg
membangun : bangun
habit : habit
fi : fi
kejadian : jadi
gak : gak
knp : knp
krn : krn
hasil : hasil
dr : dr
akumulasi : akumulasi
abainya : abai
thd : thd
spt : spt
memetik : petik
buah : buah
simalakama : simalakama
pangadilan 

Pandas Apply:   0%|          | 0/80 [00:00<?, ?it/s]

0     [akibat, muka, tembok, urus, pssi, tragedi, ka...
1     [loh, desember, udah, aja, liga, ya, kubu, fed...
2                  [bangsat, laku, tragedi, kanjuruhan]
3     [tanggap, main, persiba, tragedi, darah, stadi...
4     [statement, sikap, kriting, salah, podcast, si...
                            ...                        
75    [kejat, sangka, pasal, lalai, tragedi, kanjuru...
76    [kerumit, dunia, persepakbolaan, indonesia, sa...
77    [asisten, kapolri, bidang, operasi, asops, irj...
78    [anak, korban, tragedi, kanjuruhan, damping, p...
79    [via, naswa, sintas, tragedi, kanjuruhan, juan...
Name: tweet, Length: 80, dtype: object

##Menyimpan Hasil Tahap Preprocessing ke file .csv

In [35]:

data_abstrak.to_csv('preprocessing.csv')

# **TF**

TF(Term Frequency) : Istilah frekuensi kata dalam dokumen. Ada beberapa cara untuk menghitung frekuensi ini, dengan cara yang paling sederhana adalah dengan menghitung jumlah kata yang muncul dalam dokumen. Lalu, ada cara untuk menyesuaikan frekuensi, berdasarkan panjang dokumen, atau dengan frekuensi mentah kata yang paling sering muncul dalam dokumen.

In [36]:

from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
#Membuat Dataframe
dataTextPre = pd.read_csv('preprocessing.csv')
vectorizer = CountVectorizer(min_df=1)
bag = vectorizer.fit_transform(dataTextPre['tweet'])
dataTextPre

,Unnamed: 0,Unnamed: 0.1,tweet
0,0,0,"['akibat', 'muka', 'tembok', 'urus', 'pssi', '..."
1,1,1,"['loh', 'desember', 'udah', 'aja', 'liga', 'ya..."
2,2,2,"['bangsat', 'laku', 'tragedi', 'kanjuruhan']"
3,3,3,"['tanggap', 'main', 'persiba', 'tragedi', 'dar..."
4,4,4,"['statement', 'sikap', 'kriting', 'salah', 'po..."
...,...,...,...
75,75,75,"['kejat', 'sangka', 'pasal', 'lalai', 'tragedi..."
76,76,76,"['kerumit', 'dunia', 'persepakbolaan', 'indone..."
77,77,77,"['asisten', 'kapolri', 'bidang', 'operasi', 'a..."
78,78,78,"['anak', 'korban', 'tragedi', 'kanjuruhan', 'd..."


In [37]:
matrik_vsm=bag.toarray()
#print(matrik_vsm)
matrik_vsm.shape

(80, 482)

In [38]:
matrik_vsm[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [39]:
a=vectorizer.get_feature_names()

In [40]:
print(len(matrik_vsm[:,1]))
#dfb =pd.DataFrame(data=matrik_vsm,index=df,columns=[a])
dataTF =pd.DataFrame(data=matrik_vsm,index=list(range(1, len(matrik_vsm[:,1])+1, )),columns=[a])
dataTF

80


,abai,acara,achmadi,adil,advokasi,aff,agung,ahy,air,aj,...,via,warga,whats,wooi,ya,yakalo,yatim,yg,you,yudayana
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
dataTF.to_csv('TFIDF.csv')